In [16]:
from typing import Dict, Text
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import tensorflow as tf

import tensorflow_recommenders as tfrs

In [3]:
df_reviews = pd.read_csv("/programing/Simulasi TF/SIMULASI/ML/Dataset/reviews.csv")
df_recipes = pd.read_csv("/programing/Simulasi TF/SIMULASI/ML/Dataset/recipes.csv")


In [4]:
# df_recipes.drop(['CookTime', 'PrepTime', 'TotalTime', 'DatePublished'], axis=1, inplace=True)
df_recipes = df_recipes.rename(columns={
    'FatContent': 'Fat',
    'SaturatedFatContent': 'SaturatedFat',
    'CholesterolContent': 'Cholesterol',
    'SodiumContent': 'Sodium',
    'CarbohydrateContent	': 'Carbohydrate',
    'FiberContent': 'Fiber',
    'SugarContent': 'Sugar',
    'ProteinContent': 'Protein'
})
df_recipes.head(3)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFat,Cholesterol,Sodium,CarbohydrateContent,Fiber,Sugar,Protein,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."


In [5]:
user_counts = df_reviews.groupby('AuthorId')['RecipeId'].nunique()
single_recipe_users = user_counts[user_counts == 1].index.tolist()
recipesidss = df_reviews.loc[df_reviews['AuthorId'].isin(single_recipe_users), 'RecipeId']
recipe_counts = df_reviews.loc[df_reviews['RecipeId'].isin(recipesidss),'RecipeId'].value_counts()
len(recipe_counts)

71213

In [6]:
useless_recipes = recipe_counts[recipe_counts == 1].index.tolist()
len(useless_recipes)

11411

In [7]:
df_reviews = df_reviews[~df_reviews["RecipeId"].isin(useless_recipes)]
len(df_reviews)

1390571

In [8]:
df_recipes = df_recipes[~df_recipes["RecipeId"].isin(useless_recipes)]
len(df_recipes)

511106

In [9]:
recipesReviews = df_recipes.loc[df_recipes['RecipeId'].isin(df_reviews['RecipeId'].unique())]
len(recipesReviews)

260263

In [10]:
recipesNot_in_list = df_reviews.loc[~df_reviews['RecipeId'].isin(df_recipes['RecipeId'])]
len(recipesNot_in_list)

19

In [11]:
df_reviews = df_reviews.loc[~df_reviews['ReviewId'].isin(recipesNot_in_list['ReviewId'])]
len(df_reviews)

1390552

In [12]:
duplicates = recipesReviews[recipesReviews.duplicated(subset=['Name','AuthorId'],keep=False)]
dropped_duplicates_df = duplicates.drop_duplicates(subset=['Name','AuthorId'],keep='first')

In [13]:
temp2 = df_reviews.loc[df_reviews['RecipeId'].isin(duplicates['RecipeId'])]
temp2 = temp2.drop(temp2[temp2['RecipeId'] == 28199].index)
reviews_filterd = df_reviews.drop(index=df_reviews[df_reviews['ReviewId'].isin(temp2['ReviewId'])].index)

In [17]:
user_ids = reviews_filterd["AuthorId"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
recipe_ids = reviews_filterd["RecipeId"].unique().tolist()
recipeID2recipeID_encoded = {x: i for i, x in enumerate(recipe_ids)}
recipeID_encoded2recipeID = {i: x for i, x in enumerate(recipe_ids)}
reviews_filterd["AuthorId"] = reviews_filterd["AuthorId"].map(user2user_encoded)
reviews_filterd["RecipeId"] = reviews_filterd["RecipeId"].map(recipeID2recipeID_encoded)

num_users = len(user2user_encoded)
num_recipes = len(recipeID_encoded2recipeID)
reviews_filterd["Rating"] = reviews_filterd["Rating"].values.astype(np.float32)
# min and max ratings will be used to normalize the ratings later
min_rating = min(reviews_filterd["Rating"])
max_rating = max(reviews_filterd["Rating"])

print(
    "Number of users: {}, Number of Recipes: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_recipes, min_rating, max_rating
    )
)

Number of users: 260292, Number of Recipes: 259893, Min rating: 0.0, Max rating: 5.0


In [18]:
reviews_filterd = reviews_filterd.sample(frac=1, random_state=42)
x = reviews_filterd[["AuthorId", "RecipeId"]].values
# Normalize the targets between 0 and 1. Makes it easy to train.
y = reviews_filterd["Rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values
# Assuming training on 90% of the data and validating on 10%.
train_indices = int(0.9 * reviews_filterd.shape[0])
x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)
reviews_filterd["Rating"] = reviews_filterd["Rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values

In [19]:
pd.set_option('display.max_columns',28)
df_reviews.head(3)

,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17T16:49:59Z,2001-10-17T16:49:59Z
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z


In [20]:
pd.set_option('display.max_columns',28)
df_recipes.head(3)

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,Keywords,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,Fat,SaturatedFat,Cholesterol,Sodium,CarbohydrateContent,Fiber,Sugar,Protein,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",Frozen Desserts,"c(""Dessert"", ""Low Protein"", ""Low Cholesterol"",...","c(""4"", ""1/4"", ""1"", ""1"")","c(""blueberries"", ""granulated sugar"", ""vanilla ...",4.5,4.0,170.9,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""Chicken Thigh & Leg"", ""Chicken"", ""Poultry"",...","c(""1"", ""4"", ""2"", ""2"", ""8"", ""1/4"", ""8"", ""1/2"", ...","c(""saffron"", ""milk"", ""hot green chili peppers""...",3.0,1.0,1110.7,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""Low Protein"", ""Low Cholesterol"", ""Healthy"",...","c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."


In [52]:

# train_reviews, test_reviews = train_test_split(df_reviews, test_size=0.2, random_state=42)
# train_reviews.to_csv('/programing/Simulasi TF/SIMULASI/ML/Dataset/train_reviews.csv', index=False)
# test_reviews.to_csv('/programing/Simulasi TF/SIMULASI/ML/Dataset/test_reviews.csv', index=False)

: 

In [21]:
reviews_train = pd.read_csv("/programing/Simulasi TF/SIMULASI/ML/Dataset/train_reviews.csv")
reviews_test = pd.read_csv("/programing/Simulasi TF/SIMULASI/ML/Dataset/test_reviews.csv")

df_reviews = df_reviews.astype({'ReviewId': 'string', 'RecipeId':'string'})
reviews_train = reviews_train.astype({'ReviewId': 'string', 'RecipeId':'string'})
reviews_test = reviews_test.astype({'ReviewId': 'string', 'RecipeId':'string'})

In [22]:
uniqueUserIds = df_reviews.ReviewId.unique()
uniqueFoodIds = df_reviews.RecipeId.unique()

In [23]:
class RankingModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        self.user_embeddings = tf.keras.Sequential([
                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=uniqueUserIds, mask_token=None),
                                        # add addional embedding to account for unknow tokens
                                    tf.keras.layers.Embedding(len(uniqueUserIds)+1, embedding_dimension)
                                    ])

        self.product_embeddings = tf.keras.Sequential([
                                    tf.keras.layers.experimental.preprocessing.StringLookup(
                                        vocabulary=uniqueFoodIds, mask_token=None),
                                    # add addional embedding to account for unknow tokens
                                    tf.keras.layers.Embedding(len(uniqueFoodIds)+1, embedding_dimension)
                                    ])
        # Set up a retrieval task and evaluation metrics over the
        # entire dataset of candidates.
        self.ratings = tf.keras.Sequential([
                            tf.keras.layers.Dense(256, activation="relu"),
                            tf.keras.layers.Dense(64,  activation="relu"),
                            tf.keras.layers.Dense(1)
                              ])

    def call(self, userId, foodId):
        user_embeddings  = self.user_embeddings (userId)
        food_embeddings = self.product_embeddings(foodId)
        return self.ratings(tf.concat([user_embeddings, food_embeddings], axis=1))



In [24]:
# Build a model.
class FoodModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer   = tfrs.tasks.Ranking(
                                                    loss    =  tf.keras.losses.MeanSquaredError(),
                                                    metrics = [tf.keras.metrics.RootMeanSquaredError()])


    def compute_loss(self, features, training=False):
        rating_predictions = self.ranking_model(features["userID"], features["foodID"]  )

        return self.task( labels=features["rating"], predictions=rating_predictions)

In [26]:
train_data = tf.data.Dataset.from_tensor_slices(
{
    "userID":tf.cast(reviews_train.ReviewId.values, tf.string),
    "foodID":tf.cast(reviews_train.RecipeId.values, tf.string),
    "rating":tf.cast(reviews_train.Rating.values, tf.float32)
})

test_data = tf.data.Dataset.from_tensor_slices(
{
    "userID":tf.cast(reviews_test.ReviewId.values, tf.string),
    "foodID":tf.cast(reviews_test.RecipeId.values, tf.string),
    "rating":tf.cast(reviews_test.Rating.values, tf.float32)
})

In [27]:
tf.random.set_seed(42)

train_data = train_data.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

In [28]:
model = FoodModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.001))
cached_train = train_data.shuffle(100_000).batch(8192).cache()
cached_test = test_data.batch(4096).cache()
model.fit(cached_train, epochs=10)

Epoch 1/10
136/136 [==============================] - 108s 138ms/step - root_mean_squared_error: 4.2614 - loss: 18.1356 - regularization_loss: 0.0000e+00 - total_loss: 18.1356
Epoch 2/10
136/136 [==============================] - 7s 51ms/step - root_mean_squared_error: 3.5465 - loss: 12.5528 - regularization_loss: 0.0000e+00 - total_loss: 12.5528
Epoch 3/10
136/136 [==============================] - 7s 49ms/step - root_mean_squared_error: 2.7242 - loss: 7.4038 - regularization_loss: 0.0000e+00 - total_loss: 7.4038
Epoch 4/10
136/136 [==============================] - 7s 48ms/step - root_mean_squared_error: 2.0158 - loss: 4.0542 - regularization_loss: 0.0000e+00 - total_loss: 4.0542
Epoch 5/10
136/136 [==============================] - 7s 49ms/step - root_mean_squared_error: 1.5629 - loss: 2.4391 - regularization_loss: 0.0000e+00 - total_loss: 2.4391
Epoch 6/10
136/136 [==============================] - 7s 50ms/step - root_mean_squared_error: 1.3569 - loss: 1.8402 - regularization_loss:

In [29]:
user_rand = uniqueUserIds[200]
test_rating = {}
for m in test_data.take(10):
    test_rating[m["foodID"].numpy()]=RankingModel()(tf.convert_to_tensor([user_rand]),tf.convert_to_tensor([m["foodID"]]))

In [33]:
print("Top 10 recommended products for User {}: ".format(user_rand))
for m in sorted(test_rating, key=test_rating.get, reverse=True):
    print(df_recipes.loc[df_recipes['RecipeId'] == int(m.decode())]['Name'].item())

Top 10 recommended products for User 312: 
Honey-Herb Cornish Game Hens
Ultimate Peanut Butter Pie
Cream Soup Mix (Replacement for Cream of ___ Soups) Substitute
Reed's &quot;slap Yo' Momma They're so Good&quot;  Brownies
Creamy Chickpea &amp; Rosemary Soup
Kittencal's Chinese Chicken Balls With Sweet and Sour Sauce
My Own Coleslaw Dressing
Fresh Tomato and Spinach Pasta
Taquito-Enchilada Bake
Zesty Low-Fat Chicken Breasts
